In [14]:
%config Completer.use_jedi = False

import pandas as pd
import re
import math
from datetime import datetime
import numpy
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget


In [15]:
def processDate(string, regexp):
    result = None
    if string != 'nan':
        dates = re.findall(regexp, string)
        if len(dates) != 0:
            result = dates[0]
    return result

In [16]:
def days_between(d1, d2, format_first, format_second):
    try:
        d1 = datetime.strptime(d1, format_first)
        d2 = datetime.strptime(d2, format_second)
    except ValueError:
        return None
    return abs((d2 - d1).days)

In [23]:
def filterData(priziv_from_beginning, smert_from_priziv, age_at_priziv):
    if priziv_from_beginning is None or smert_from_priziv is None or age_at_priziv is None:
        return False
    if priziv_from_beginning > 1460 or smert_from_priziv > 1460 or age_at_priziv > 90 * 365:
        return False
    return True

In [31]:
def processPerson(index):
    regex_month_year =  '\d{2}.\d{4}'
    priziv = processDate(str(data['data_i_mesto_priziva'][index]), regex_month_year)
    birth = processDate(str(data['date_birth'][index]), '\d{4}')
    smert = processDate(str(data['data_vibitiya'][0]), regex_month_year)
    
    if priziv is not None and birth is not None and smert is not None:
        priziv_from_beginning = days_between(priziv, '22.06.1941', "%m.%Y", "%d.%m.%Y")
        smert_from_priziv = (days_between(priziv, smert, "%m.%Y", "%m.%Y"))
        age_at_priziv = (days_between(birth, priziv, '%Y', "%m.%Y"))
        if filterData(priziv_from_beginning, smert_from_priziv, age_at_priziv):
            x1.append(priziv_from_beginning)
            x2.append(age_at_priziv)
            y.append(smert_from_priziv)
            

In [32]:
data = pd.read_csv('data.tsv', sep = '\t')
x1 = list()
x2 = list()
y = list()
for index in data.index:
    processPerson(index)
X = numpy.array([x1,x2])
priznaki = X.T
y = numpy.array(y).T
print(x1[5], end = ' ')
print(x2[5], end = ' ')
print(y[5], end = ' ')



82 7395 456 

In [33]:
model = LinearRegression()
model.fit(priznaki, y)
model.predict(numpy.array([[85, 7400]]))

array([398.30563668])

In [34]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(numpy.array(x1), numpy.array(x2), numpy.array(y))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …